# Continue writing functions to find the closest point on a route to our actual data point

Started by Nathaniel on Saturday, June 15, 2019

In [1]:
%load_ext autoreload
%autoreload 2

!date
!whoami

import numpy as np
import pandas as pd

Wed Jun 19 20:57:32 PDT 2019
ndbs


## Import my closest point module and read in a GTFS `shapes.txt` file

In [3]:
import find_closest_route_point as f

In [4]:
!ls ../data/source/gtfs_20180815/

agency.txt          calendar.txt        fare_rules.txt      stop_times.txt
block.txt           calendar_dates.txt  routes.txt          stops.txt
block_trip.txt      fare_attributes.txt shapes.txt          trips.txt


In [5]:
shapes_df = pd.read_csv('../data/source/gtfs_20180815/shapes.txt')
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,10002011,47.612137,-122.281769,1,0.0
1,10002011,47.612144,-122.281784,2,5.8
2,10002011,47.612148,-122.281830,3,13.5
3,10002011,47.612141,-122.281853,4,22.0
4,10002011,47.612102,-122.281921,5,45.0


# Testing, testing...

In [6]:
def get_two_things(x):
    return x+1, x-1

a = [1,2,3]
b = [get_two_things(x) for x in a]
b

[(2, 0), (3, 1), (4, 2)]

In [7]:
c = 3
c = -c
c

-3

## Test my "point data" functions

In [169]:
shape_id = 10002011
shape_pt_sequence = 6
point_data = f.get_shape_point_data(shapes_df, shape_id, shape_pt_sequence)
point_data

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
5,10002011,47.612129,-122.28199,6,64.2


In [11]:
adjacent_point_data = f.get_adjacent_shape_point_data(shapes_df, point_data.index[0])
adjacent_point_data

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
4,10002011,47.612102,-122.281921,5,45.0
6,10002011,47.612160,-122.282021,7,75.4


## Create a fake vehicle location to test with

In [13]:
1e-5

1e-05

In [89]:
# ['shape_pt_lat', 'shape_pt_lon']
point_coords = point_data[['shape_pt_lon', 'shape_pt_lat']].values
point_coords

array([[-122.28199  ,   47.6121292]])

In [139]:
veh_coords = np.random.normal(point_coords, 1e-5)
veh_coords

array([[-122.2819883,   47.6121302]])

In [140]:
adjacent_point_coords = adjacent_point_data[['shape_pt_lon', 'shape_pt_lat']].values
adjacent_point_coords

array([[-122.281921 ,   47.6121025],
       [-122.282021 ,   47.6121597]])

## Test point projection function with broadcasting over both adjacent points

Haha! I didn't initially design it to work on multiple line segments at once, but all I had to do was add `axis=1` to the distance calculation, because `numpy` is smart like that.

Note, however, that it does NOT work if we replace the numpy arrays with pandas dataframes.

In [141]:
f.get_projection_and_dist_ratio(veh_coords, point_coords, adjacent_point_coords)

(array([[-122.28198891,   47.61212941],
        [-122.28199049,   47.61212896]]), array([[ 0.01575062, -0.00781741]]))

In [142]:
closest_point, dist_ratio = f.get_projection_and_dist_ratio(veh_coords, point_coords, adjacent_point_coords)

In [143]:
direction = adjacent_point_coords - point_coords
direction

array([[ 6.90e-05, -2.67e-05],
       [-3.10e-05,  3.05e-05]])

In [144]:
np.sum(direction**2, axis=1) # Check that axis=1 is correct for distance calculation

array([5.47389e-09, 1.89125e-09])

## Compute squared distance from vehicle to both projected points

In [145]:
veh_coords

array([[-122.2819883,   47.6121302]])

In [146]:
closest_point

array([[-122.28198891,   47.61212941],
       [-122.28199049,   47.61212896]])

In [147]:
veh_coords-closest_point

array([[6.13785531e-07, 7.95234641e-07],
       [2.18884736e-06, 1.24239043e-06]])

In [148]:
dist_squared = np.sum((veh_coords-closest_point)**2, axis=1)
dist_squared

array([1.00913081e-12, 6.33458674e-12])

## Get the point and distance ratio corresponding to the minimum distance

In [149]:
np.argmin(dist_squared)

0

In [150]:
dist_squared[1:]

array([6.33458674e-12])

In [151]:
np.argmin(dist_squared[1:])

0

In [152]:
min_index = np.argmin(dist_squared)
closest_point[min_index]

array([-122.28198891,   47.61212941])

In [153]:
dist_ratio # Wrong shape leads to Index out of bounds if you do dist_ratio[min_index] with min_index=1

array([[ 0.01575062, -0.00781741]])

In [154]:
dist_ratio.reshape(2,)

array([ 0.01575062, -0.00781741])

In [155]:
dist_ratio.reshape(2,)[min_index]

0.015750618373767625

## Reset `closest_point` and `dist_ratio` to be a single point and a single number instead of potentially a pair of points and pair of numbers

In [156]:
closest_point, dist_ratio = closest_point[min_index], dist_ratio.reshape(2,)[min_index]

In [157]:
print(closest_point, dist_ratio)

[-122.28198891   47.61212941] 0.015750618373767625


## Determine if the closest shape point is ahead of or behind the vehicle on the route

Get the `shape_pt_sequence` number for the other endpoint of the segment, and compare it to that of the closest shape point (stored in `point_data`). We can find the other endpoint of the segment by using the `min_index` that we found.

* If the other endpoint comes after the original point (the closest shape point), then the original point is behind the vehicle. In this case we leave `dist_ratio` alone because we'll need to add to shape distance to the original point.
* If the other endpoint comes before the original point, then the original point is ahead of the vehicle. In this case, we negate `dist_ratio` becasuse we'll need to subtract from the shape distance to the original point.

## Actually, NO!!! We do NOT need to do this! The signs will be automatically correct.

That's the magic of algebra...

In [158]:
segment_end_data = adjacent_point_data.iloc[min_index]
segment_end_data

shape_id               1.000201e+07
shape_pt_lat           4.761210e+01
shape_pt_lon          -1.222819e+02
shape_pt_sequence      5.000000e+00
shape_dist_traveled    4.500000e+01
Name: 4, dtype: float64

In [159]:
point_data

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
5,10002011,47.612129,-122.28199,6,64.2


In [160]:
segment_end_data.shape_dist_traveled

45.0

In [161]:
point_data.iloc[0].shape_dist_traveled

64.2

### Ok, let's try this in a different order

In [162]:
closest_shape_dist = point_data.iloc[0].shape_dist_traveled
next_shape_dist = adjacent_point_data.iloc[min_index].shape_dist_traveled
print(closest_shape_dist, next_shape_dist)

64.2 45.0


In [165]:
# if next_shape_dist < closest_shape_dist:
#     dist_ratio = -dist_ratio
# dist_ratio

0.015750618373767625

In [166]:
shape_dist_traveled = closest_shape_dist + dist_ratio * (next_shape_dist - closest_shape_dist)
shape_dist_traveled

63.89758812722366

## Make sure my function returns the same closest route point and shape distance traveled

In [175]:
veh_lon, veh_lat = veh_coords.reshape(2,)
f.find_closest_point_on_route(shapes_df, shape_id, veh_lat, veh_lon, shape_pt_sequence)

(array([-122.28198891,   47.61212941]), 63.89758812722366)

In [176]:
# Check that they're equal. Woo hoo!
all([np.allclose(closest_point,_[0]), shape_dist_traveled, _[1]])

True